In [2]:
import tatqa_utils
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [3]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [4]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [5]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [6]:
def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code(llm, question, table)
exec_code(code, table)

(14.065510597302506, '%')

In [7]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)

remove_thousand_separators('alma 123,432')

'alma 123432'

In [8]:
for idx, item in devdf.iterrows():
    if item['table']['uid'] == '68bafc82-b795-4c7b-9506-e901223c3526':        
        print("==========")
        for q in item['questions']:            
            if q['uid'] == '1e20b997-cb3b-4bc5-93e6-b45dbe945fb5':
                table = item['table']['table']
                table = [remove_thousand_separators(cell) for row in table for cell in row]
                if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:                     
                    code = gen_code(llm, q, table)
                    for l in table:
                        print(l)
                    print("\n", q['question'],"\n")
                    print(code)
                    r = exec_code(code, table)
                    print(r, q['answer'], q['scale'])



Years Ended December 31,


2019
2018
2017
Americas:



United States
$614493
$668580
$644870
The Philippines
250888
231966
241211
Costa Rica
127078
127963
132542
Canada
99037
102353
112367
El Salvador
81195
81156
75800
Other
123969
118620
118853
Total Americas
1296660
1330638
1325643
EMEA:



Germany
94166
91703
81634
Other
223847
203251
178649
Total EMEA
318013
294954
260283
Total Other
89
95
82

$1614762
$1625687
$1586008

 What was the change in Total Other in 2018 from 2017? 

def run():
    return (13, 'thousand')
(13, 'thousand') 13 thousand


In [1]:
res[0]

NameError: name 'res' is not defined